In [10]:
# настройка kaggle 
! gdown --id 1rvsKiuQjWXUkRASRZv3qSSSCxTgKU_w5 # load kaggle.json
! rm -rf ~/.kaggle
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1rvsKiuQjWXUkRASRZv3qSSSCxTgKU_w5
To: /content/kaggle.json
100% 64.0/64.0 [00:00<00:00, 110kB/s]


In [14]:
# загрузка датасетов
! kaggle competitions download -c home-credit-default-risk
! unzip home-credit-default-risk.zip

home-credit-default-risk.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  home-credit-default-risk.zip
  inflating: HomeCredit_columns_description.csv  
  inflating: POS_CASH_balance.csv    
  inflating: application_test.csv    
  inflating: application_train.csv   
  inflating: bureau.csv              
  inflating: bureau_balance.csv      
  inflating: credit_card_balance.csv  
  inflating: installments_payments.csv  
  inflating: previous_application.csv  
  inflating: sample_submission.csv   


In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [38]:
# описание сnолбцов для application_{train|test}.csv	
columns_description_data = pd.read_csv("/content/HomeCredit_columns_description.csv", encoding = "ISO-8859-1")
columns_description_data = columns_description_data[columns_description_data["Table"] == "application_{train|test}.csv"]
columns_description_data = columns_description_data.drop(["Table", "Unnamed: 0"], axis=1)

columns_description_data.sample(6)

,Row,Description,Special
9,AMT_ANNUITY,Loan annuity,NaN
100,FLAG_DOCUMENT_6,Did client provide document 6,NaN
75,YEARS_BUILD_MEDI,Normalized information about building where th...,normalized
7,AMT_INCOME_TOTAL,Income of the client,NaN
42,EXT_SOURCE_2,Normalized score from external data source,normalized
38,REG_CITY_NOT_WORK_CITY,Flag if client's permanent address does not ma...,NaN


In [36]:
columns_description_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122 entries, 0 to 121
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Row          122 non-null    object
 1   Description  122 non-null    object
 2   Special      56 non-null     object
dtypes: object(3)
memory usage: 3.8+ KB


In [43]:
columns_description_data.describe()

,Row,Description,Special
count,122,122,56
unique,122,73,4
top,SK_ID_CURR,Normalized information about building where th...,normalized
freq,1,47,50


In [42]:
columns_description_data["Special"].unique()

array([nan, 'normalized ', 'time only relative to the application',
       'rounded', 'normalized'], dtype=object)

In [25]:
application_train_data = pd.read_csv("/content/application_train.csv")
application_test_data = pd.read_csv("/content/application_test.csv")

print(application_test_data.info())
print(application_train_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 121 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(16)
memory usage: 45.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB
None


In [45]:
application_test_data.sample(6)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
27460,300363,Cash loans,F,Y,Y,0,270000.0,779409.0,28125.0,643500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0
35376,358937,Cash loans,F,N,Y,0,157500.0,360000.0,23134.5,360000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
5926,143360,Cash loans,F,Y,N,1,315000.0,900000.0,43429.5,900000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
15464,211550,Cash loans,F,N,N,0,99000.0,450000.0,14517.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
14988,207976,Cash loans,F,N,Y,0,112500.0,573408.0,29407.5,495000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
43806,419750,Cash loans,F,N,N,2,202500.0,479700.0,49153.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,2.0,0.0


In [ ]:
application_test_data.dtypes

In [47]:
application_test_data.describe(include="all")

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,48744.000000,48744,48744,48744,48744,48744.000000,4.874400e+04,4.874400e+04,48720.000000,4.874400e+04,...,48744.000000,48744.0,48744.0,48744.0,42695.000000,42695.000000,42695.000000,42695.000000,42695.000000,42695.000000
unique,NaN,2,2,2,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Cash loans,F,N,Y,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,48305,32678,32311,33658,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,277796.676350,NaN,NaN,NaN,NaN,0.397054,1.784318e+05,5.167404e+05,29426.240209,4.626188e+05,...,0.001559,0.0,0.0,0.0,0.002108,0.001803,0.002787,0.009299,0.546902,1.983769
std,103169.547296,NaN,NaN,NaN,NaN,0.709047,1.015226e+05,3.653970e+05,16016.368315,3.367102e+05,...,0.039456,0.0,0.0,0.0,0.046373,0.046132,0.054037,0.110924,0.693305,1.838873
min,100001.000000,NaN,NaN,NaN,NaN,0.000000,2.694150e+04,4.500000e+04,2295.000000,4.500000e+04,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,188557.750000,NaN,NaN,NaN,NaN,0.000000,1.125000e+05,2.606400e+05,17973.000000,2.250000e+05,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,277549.000000,NaN,NaN,NaN,NaN,0.000000,1.575000e+05,4.500000e+05,26199.000000,3.960000e+05,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,367555.500000,NaN,NaN,NaN,NaN,1.000000,2.250000e+05,6.750000e+05,37390.500000,6.300000e+05,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000


In [ ]:
# пропуски
application_test_data.isnull().sum()